In [4]:
from html_parser import BSParser
from report import Report

In [7]:
r = Report("..", "html", "data/Report.html")
h = BSParser("data/Report.html","URL")

In [8]:
print(h.get_keywords())
print(h.get_date())
print(h.get_source())

['opencitations', ' overview']
18/11/2019
URL


/Users/ivan.heibi/ivanhb.github.io/edu/model/script/html_parser.py:27: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 27 of the file /Users/ivan.heibi/ivanhb.github.io/edu/model/script/html_parser.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(f)


In [ ]:
r.add_entry({"attribute":{"date":"1","source":"2"},"relation":{"@hasKeyword":"3"}})

In [ ]:
l = "2019-11-18 10:21:38 +0000"[0:10].split("-")

In [ ]:
date = l[2]+"/"+l[1]+"/"+l[0]

In [ ]:
print(date)

In [ ]:
## Build the model index

In [125]:
from model import Model
from bs4 import BeautifulSoup
import codecs
import re
import os

class Graphml(object):
    
    OBJECT_PROP_KEY = "@"
    SUBCLASS_KEY = "@isSubclassOf"
    DATATYPES = {
        "string":{"default":""},
        "int":{"default":-1},
        "datetime":{"default":-1},
        "url":{"default":-1}
    }

    def __init__(self, input_file_path): 
        self.nodes = []
        self.edges = []
        with codecs.open(input_file_path, 'r', encoding='utf8') as f:
            self.raw = f.read()
            self.soup = BeautifulSoup(self.raw)

    def get_edges(self):
        res = []
        edges = self.find_all("edge",{})
        for e in edges:
            bs_e = BeautifulSoup(str(e))
            res.append({
                "id": bs_e.find("edge")["id"],
                "source": bs_e.find("edge")["source"],
                "target": bs_e.find("edge")["target"],
                "value": bs_e.find("y:edgelabel").contents
            })

        for i in range(0,len(res)): 
            str_on_edge = res[i]["value"][0].split(",")[0]
            res[i]["value"] = str_on_edge.replace("\n",";")
        return res

    def get_nodes(self):
        res = []
        nodes = self.find_all("node",{})
        for n in nodes:
            bs_n = BeautifulSoup(str(n))
            res.append({
                "id": bs_n.find("node")["id"],
                "value": "".join(bs_n.find("y:nodelabel").contents)
            })
        return res

    def find_all(self,tag,att):
        return self.soup.findAll(tag,att)

    
    def normalize_into_owl_formalism(self):
        OWL_GRAPH = {
            "classes": [],
            "datatypes": [],
            "data_properties": [],
            "object_properties": []
        }
        
        self.nodes = self.get_nodes()
        self.edges = self.get_edges()
        
        #Classes and Datatypes
        for n in self.nodes:
            if(n["value"][0].isupper()):
                OWL_GRAPH["classes"].append(n)
            else:
                OWL_GRAPH["datatypes"].append(n)
        
        # Object/Data properties  
        obj_props = []
        data_props = []
        for e in self.edges:
            edge_type = "dataProp"
            if(e["value"][0] == self.OBJECT_PROP_KEY):
                edge_type = "objProp"
                    
            e["value"] = e["value"].split(";")
            if edge_type == "dataProp":
                OWL_GRAPH["data_properties"].append(e)
            elif edge_type == "objProp":    
                OWL_GRAPH["object_properties"].append(e)
                
        return OWL_GRAPH
    
    
    # JSON Jago index,
    def build_jago_index(self,owl_gr, path = None):
        
        def _find_elem(items_list,att,val):
            for item in items_list:
                if item[att] == val:
                    return item
            return -1
        
        def _build_tree(classes_index, class_val, tree):
            if SUBCLASS_KEY in classes_index[class_val]["object_properties"]:
                sub_class_val = classes_index[class_val]["object_properties"][SUBCLASS_KEY]
                tree.append(sub_class_val)
                return _build_tree(classes_index, sub_class_val, tree)
            else:
                return tree
    
    
        index_prefixes = {}
        classes_index = {}
        for c in owl_gr["classes"]:
            classes_index[c["value"]]= {
                "prefix":None,
                "last_id": 0,
                "data_properties": {},
                "object_properties": {}
            }
            #check data properties
            for dp in owl_gr["data_properties"]:
                    if (dp["source"] == c["id"]):
                        for dp_val in dp["value"]:
                            default_value = -1
                            if dp_val in self.DATATYPES:
                                default_value = self.DATATYPES[dp["value"]]
                            classes_index[c["value"]]["data_properties"][dp_val] = default_value
            
            #check object properties
            for op in owl_gr["object_properties"]:
                    if (op["source"] == c["id"]):            
                        for op_val in op["value"]:
                            target_node = _find_elem(owl_gr["classes"],"id",op["target"])
                            classes_index[c["value"]]["object_properties"][op_val] = target_node["value"]
            
            #define prefix
            pref = c["value"][:2].lower() 
            i = 2
            while (pref in index_prefixes):
                pref = pref+c["value"][i].lower() 
                i += 1
            index_prefixes[pref] = True
            classes_index[c["value"]]["prefix"] = pref
        
        #in case a path where to build the structure is specified
        if path != None:
            #a local dir
            if not path.strtsWith("http"):
                model_path = path+"model/"
                os.mkdir(model_path)
                
                #save the index.json
                with open(classes_index+'index.json', 'w') as f:
                    json.dump(data, f)
                
                #now all the structure
                src_path = path+"model/src/"
                os.mkdir(src_path)
                for class_k in classes_index:
                    tree = _build_tree(classes_index, class_k, [class_k])
                    
        return classes_index  

In [126]:
graphml = Graphml("data/modelv2.graphml")

In [127]:
owl_gr = graphml.normalize_into_owl_formalism();

In [128]:
graphml.build_jago_index(owl_gr)

{'Activity': {'prefix': 'ac',
  'last_id': 0,
  'data_properties': {'title': -1,
   'type': -1,
   'location': -1,
   'webpage': -1,
   'start_date': -1,
   'end_date': -1},
  'object_properties': {'@hasKeyword': 'Keyword'}},
 'Conference': {'prefix': 'co',
  'last_id': 0,
  'data_properties': {},
  'object_properties': {'@isSubclassOf': 'Activity'}},
 'Teaching': {'prefix': 'te',
  'last_id': 0,
  'data_properties': {'course': -1, 'host': -1},
  'object_properties': {'@isSubclassOf': 'Activity'}},
 'Training': {'prefix': 'tr',
  'last_id': 0,
  'data_properties': {'type': -1},
  'object_properties': {'@isSubclassOf': 'Activity'}},
 'Seminary': {'prefix': 'se',
  'last_id': 0,
  'data_properties': {'presenter': -1},
  'object_properties': {'@isSubclassOf': 'Conference'}},
 'Publication': {'prefix': 'pu',
  'last_id': 0,
  'data_properties': {'reference': -1, 'referenceFormat': -1, 'author': -1},
  'object_properties': {'@isSubclassOf': 'Product'}},
 'Code': {'prefix': 'cod',
  'last_id